# Bird Species Classier 
Built using the [Kaggle dataset](https://www.kaggle.com/gpiosenka/100-bird-species) by [Gerry](https://www.kaggle.com/gpiosenka)

Project By: [Uzair](https://github.com/Uzair05)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf